<a href="https://colab.research.google.com/github/peremartra/fairness-pruning/blob/main/notebooks/02_Evaluate_MBBQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fairness Pruning Research - MBBQ (EsBBQ) Evaluation
## 02 - Multilingual BBQ Benchmark for Spanish Bias Detection

### Establishing Bias Performance using EsBBQ (MBBQ) for Spanish Bias Mitigation Research
by [Pere Martra](https://github.com/peremartra)

[![GitHub](https://img.shields.io/badge/⭐_Star-OptiPFair-orange?logo=github&logoColor=white)](https://github.com/peremartra/optipfair)
[![PyPI](https://img.shields.io/pypi/v/optipfair?logo=python&logoColor=white&label=v)](https://pypi.org/project/optipfair/)

**Repository:** [github.com/peremartra/fairness-pruning](https://github.com/peremartra/fairness-pruning)

---

**Colab Environment:** GPU L4 or A100

**Models to Evaluate:**
* Llama-3.2-1B (base)
* Llama-3.2-3B (base)
* Salamandra-2B (base)

**EsBBQ Categories:**
* Age, Disability Status, Gender, LGBTQIA+, Nationality
* Physical Appearance, Race/Ethnicity, Religion, SES, Spanish Region

---

**Note:** This notebook evaluates ONLY base models (no pruning applied) on Spanish bias benchmarks. For English BBQ evaluation, see `02_Evaluate_BBQ.ipynb`.

---

# 1. Setup & Installation

In [ ]:
# Install required libraries
!pip install -q optipfair
!pip install -q lm-eval
!pip install -q langdetect

In [ ]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

In [ ]:
# Download EsBBQ task YAML files from GitHub repository
import os
os.makedirs('custom_tasks/esbbq', exist_ok=True)

!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_age.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_disabilitystatus.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_gender.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_lgbtqia.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_nationality.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_physicalappearance.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_raceethnicity.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_religion.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_ses.yaml
!wget -q -P custom_tasks/esbbq https://raw.githubusercontent.com/peremartra/fairness-pruning/main/custom_tasks/esbbq/esbbq_spanishregion.yaml

# Verify downloads
yaml_count = len([f for f in os.listdir('custom_tasks/esbbq') if f.endswith('.yaml')])
print(f"✅ Downloaded {yaml_count} EsBBQ YAML files")

In [ ]:
# Set LMEVAL_INCLUDE_PATH to load custom EsBBQ tasks
import os
os.environ["LMEVAL_INCLUDE_PATH"] = "/content/custom_tasks/esbbq"
print(f"✅ LMEVAL_INCLUDE_PATH set to: {os.environ['LMEVAL_INCLUDE_PATH']}")

# 2. Helper Functions

Utility functions for automatic checkpoint path generation and model size detection.

In [ ]:
import re
import os

def get_model_size(model_name: str) -> str:
    """Extract model size identifier from HuggingFace model name.

    Examples:
        "meta-llama/Llama-3.2-1B" → "1b"
        "meta-llama/Llama-3.2-3B-Instruct" → "3b_instruct"
        "BSC-LT/salamandra-2b" → "2b"
    """
    match = re.search(r'(\d+\.?\d*)[Bb]', model_name)
    if not match:
        return "unknown"

    size = match.group(1).replace('.', '_') + "b"
    if "instruct" in model_name.lower():
        size += "_instruct"

    return size.lower()

def get_checkpoint_path(model_name: str, base_dir: str) -> str:
    """Generate checkpoint path with size-based subdirectory.

    Args:
        model_name: Full HuggingFace model identifier
        base_dir: Base directory for checkpoints

    Returns:
        Full path to checkpoint file
    """
    model_size = get_model_size(model_name)
    safe_name = model_name.replace('/', '_').replace('-', '_').lower()
    checkpoint_dir = os.path.join(base_dir, model_size)
    os.makedirs(checkpoint_dir, exist_ok=True)
    return os.path.join(checkpoint_dir, f"{safe_name}.json")

# Test with example models
print("Testing helper functions:")
print("-" * 70)
test_models = [
    "BSC-LT/salamandra-2b",
    "meta-llama/Llama-3.2-1B",
    "meta-llama/Llama-3.2-3B"
]
for model_id in test_models:
    size = get_model_size(model_id)
    print(f"{model_id:<50} → {size}")
print("-" * 70)

# 3. Configuration & Evaluation Plan (MBBQ/EsBBQ)

Configure paths, select EsBBQ tasks, and list the models we will evaluate.

In [ ]:
import torch
import json
import pandas as pd
from datetime import datetime
from pathlib import Path
import logging
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display
from utils import (
    EXPERIMENT_CONFIG,
    run_robust_evaluation,
    load_or_create_model,
    clear_gpu_cache,
    format_results_table,
    get_model_stats,
 )

# Paths (Drive recommended in Colab)
CHECKPOINT_BASE_DIR = "/content/drive/MyDrive/fair_pruning/checkpoints_mbbq"
RESULTS_DIR = "/content/drive/MyDrive/fair_pruning/results"
Path(CHECKPOINT_BASE_DIR).mkdir(parents=True, exist_ok=True)
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

# EsBBQ (MBBQ) task list - all categories in one group (0-shot)
# Using task group name that encompasses all EsBBQ subtasks
MBBQ_TASKS = [
    {"name": "esbbq_age", "num_fewshot": 0},
    {"name": "esbbq_disabilitystatus", "num_fewshot": 0},
    {"name": "esbbq_gender", "num_fewshot": 0},
    {"name": "esbbq_lgbtqia", "num_fewshot": 0},
    {"name": "esbbq_nationality", "num_fewshot": 0},
    {"name": "esbbq_physicalappearance", "num_fewshot": 0},
    {"name": "esbbq_raceethnicity", "num_fewshot": 0},
    {"name": "esbbq_religion", "num_fewshot": 0},
    {"name": "esbbq_ses", "num_fewshot": 0},
    {"name": "esbbq_spanishregion", "num_fewshot": 0}
]

# De-duplicate models from EXPERIMENT_CONFIG
unique_models = list(dict.fromkeys([cfg["base_model"] for cfg in EXPERIMENT_CONFIG]))

logging.getLogger("lm_eval").setLevel(logging.INFO)

print(f"{'='*70}")
print("📊 EVALUATION PLAN: MBBQ (EsBBQ) - Spanish Bias Benchmark")
print(f"{'='*70}")
print(f"Checkpoints: {CHECKPOINT_BASE_DIR}")
print(f"Results: {RESULTS_DIR}")
print(f"Tasks: {[task['name'] for task in MBBQ_TASKS]}")
print(f"Limit per dataset: 100 (quick test mode)")
print("Models:")
for m in unique_models:
    print(f" - {m}")
print(f"{'='*70}\n")

# 4. Run MBBQ Evaluation

Evaluate each base model on EsBBQ tasks with checkpoint/resume and raw result saving.

In [ ]:
print(f"\n{'='*70}")
print("🚀 STARTING MBBQ (EsBBQ) EVALUATION (Hugging Face Native Loading)")
print(f"{'='*70}\n")

all_model_results = {}

for idx, model_id in enumerate(unique_models, 1):
    print(f"\n{'='*70}")
    print(f"📊 MODEL {idx}/{len(unique_models)}: {model_id}")
    print(f"{'='*70}\n")
    try:
        # Generate checkpoint path
        checkpoint_path = get_checkpoint_path(model_id, CHECKPOINT_BASE_DIR)

        # Load model directly from Hugging Face
        print(f"📥 Loading directly from Hugging Face Hub...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

        tokenizer = AutoTokenizer.from_pretrained(model_id)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        print("✅ Model loaded successfully (Native HF)")

        # Show statistics
        stats = get_model_stats(model)
        print(f"📈 Params: {stats['total_parameters']:,} | Size: {stats['size_gb']:.2f} GB")
        print(f"📁 Checkpoint: {checkpoint_path}\n")

        # Run evaluation with limit=100 for quick testing
        from utils import model_evaluation
        
        # Determine raw results directory
        raw_results_dir = os.path.join(
            os.path.dirname(checkpoint_path),
            "results", "lm_evals"
        )
        os.makedirs(raw_results_dir, exist_ok=True)
        
        print(f"📊 Running evaluation with limit=100 per task...")
        print(f"💾 Raw results will be saved to: {raw_results_dir}\n")
        
        results = model_evaluation(
            model_obj=model,
            tokenizer=tokenizer,
            tasks=MBBQ_TASKS,
            limit=100,
            save_raw_results=True,
            raw_results_dir=raw_results_dir
        )
        
        # Save results to checkpoint format
        checkpoint_data = {
            "metadata": {
                "model_name": model_id,
                "started_at": datetime.now().isoformat(),
                "completed": True,
                "completed_at": datetime.now().isoformat()
            },
            "results": results,
            "pending_tasks": [],
            "failed_tasks": []
        }
        
        with open(checkpoint_path, 'w') as f:
            json.dump(checkpoint_data, f, indent=2)
        
        all_model_results[model_id] = results
        print(f"\n✅ Completed: {model_id}")
        print("Results Preview (first few tasks):")
        preview_results = {k: v for i, (k, v) in enumerate(results.items()) if i < 3}
        print(format_results_table(preview_results))

    except Exception as e:
        print(f"\n❌ ERROR evaluating {model_id}: {e}")
        import traceback
        traceback.print_exc()
        if 'model' in locals(): del model
        if 'tokenizer' in locals(): del tokenizer
        clear_gpu_cache()
        continue

    # Memory cleanup
    del model
    del tokenizer
    clear_gpu_cache()

print(f"\n{'='*70}")
print(f"✅ MBBQ EVALUATION COMPLETE: {len(all_model_results)}/{len(unique_models)} models")
print(f"{'='*70}\n")

# 5. Consolidate MBBQ Results

Load checkpoint files, flatten metrics, and export combined MBBQ results in CSV/JSON format.

In [ ]:
import glob

def flatten_metrics(metrics, prefix=''):
    """Recursively flatten nested metric dictionaries."""
    flat = {}
    for k, v in metrics.items():
        if isinstance(v, dict):
            flat.update(flatten_metrics(v, prefix=f"{prefix}{k}_"))
        else:
            flat[f"{prefix}{k}"] = v
    return flat

print(f"{'='*70}")
print(" CONSOLIDATING MBBQ RESULTS")
print(f"{'='*70}\n")

checkpoint_files = glob.glob(f"{CHECKPOINT_BASE_DIR}/**/*.json", recursive=True)
print(f"Total checkpoint JSONs found: {len(checkpoint_files)}")

consolidated_data = []

for json_path in sorted(checkpoint_files):
    # Skip raw lm-eval dumps (only process model checkpoints)
    if "lm_evals" in json_path:
        continue
    
    try:
        with open(json_path, "r") as f:
            data = json.load(f)

        metadata = data.get("metadata", {})
        model_name = metadata.get("model_name", "Unknown")

        # Extract model size
        model_size = get_model_size(model_name)

        results = data.get("results", {})
        if not results:
            continue

        for task_name, metrics in results.items():
            row = {
                "model": model_name,
                "model_size": model_size,
                "task": task_name,
            }
            row.update(flatten_metrics(metrics))
            consolidated_data.append(row)

    except Exception as e:
        print(f"   ⚠️ Error reading {json_path}: {e}")

if consolidated_data:
    df = pd.DataFrame(consolidated_data)
    df = df.sort_values(by=["model", "task"]).reset_index(drop=True)
    
    print(f"\n📊 Consolidated {len(df)} task results from {df['model'].nunique()} models")
    display(df.head(10))

    # Save timestamped and latest versions
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = f"{RESULTS_DIR}/base_models_mbbq_results_{timestamp}.csv"
    latest_csv = f"{RESULTS_DIR}/base_models_mbbq_results_latest.csv"
    json_path = f"{RESULTS_DIR}/base_models_mbbq_results_{timestamp}.json"

    df.to_csv(csv_path, index=False)
    df.to_csv(latest_csv, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print("\n💾 MBBQ results saved:")
    print(f"   {csv_path}")
    print(f"   {latest_csv}")
    print(f"   {json_path}")
else:
    print("⚠️ No MBBQ results found to consolidate.")

# 6. Summary Analysis

Quick per-model accuracy summary for MBBQ and file inventory.

In [ ]:
# Generate summary statistics
summary_df = None
if 'df' in locals() and not df.empty:
    summaries = []
    for model_name, model_df in df.groupby('model'):
        acc_series = pd.to_numeric(model_df.get('accuracy'), errors='coerce').dropna()
        summaries.append({
            "model": model_name,
            "model_size": model_df.get('model_size', pd.Series(['unknown'])).iloc[0],
            "avg_accuracy": acc_series.mean() if len(acc_series) else None,
            "tasks_completed": len(model_df),
            "categories_evaluated": len(model_df['task'].unique())
        })

    summary_df = pd.DataFrame(summaries)
    summary_df = summary_df.sort_values("model").reset_index(drop=True)
    
    print(f"{'='*70}")
    print("📈 MBBQ SUMMARY")
    print(f"{'='*70}")
    print(summary_df.to_string(index=False, float_format="%.4f"))

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    summary_csv = f"{RESULTS_DIR}/base_models_mbbq_summary_{timestamp}.csv"
    summary_df.to_csv(summary_csv, index=False)
    print(f"\n💾 Summary saved: {summary_csv}\n")
else:
    print("No consolidated data available for summary.")

# List all generated files
print(f"{'='*70}")
print("📁 GENERATED FILES")
print(f"{'='*70}")

if 'csv_path' in locals() and os.path.exists(csv_path):
    print(f"  ✅ {csv_path}")
if 'latest_csv' in locals() and os.path.exists(latest_csv):
    print(f"  ✅ {latest_csv}")
if 'json_path' in locals() and os.path.exists(json_path):
    print(f"  ✅ {json_path}")
if 'summary_csv' in locals() and os.path.exists(summary_csv):
    print(f"  ✅ {summary_csv}")

print("\nCheckpoints (first 10):")
if 'checkpoint_files' in locals():
    model_checkpoints = [f for f in sorted(checkpoint_files) if "lm_evals" not in f]
    for f in model_checkpoints[:10]:
        print(f"  ✅ {f}")
    if len(model_checkpoints) > 10:
        print(f"  ... and {len(model_checkpoints) - 10} more")

print("\nRaw LM-Eval dumps (first 10):")
if 'checkpoint_files' in locals():
    raw_dumps = [f for f in sorted(checkpoint_files) if "lm_evals" in f]
    for f in raw_dumps[:10]:
        print(f"  ✅ {f}")
    if len(raw_dumps) > 10:
        print(f"  ... and {len(raw_dumps) - 10} more")

print(f"{'='*70}")

# 7. Bias Analysis by Category

Analyze bias metrics across EsBBQ categories for each model.

In [ ]:
if 'df' in locals() and not df.empty:
    print(f"{'='*70}")
    print("📊 BIAS ANALYSIS BY CATEGORY")
    print(f"{'='*70}\n")
    
    # Extract category from task name (e.g., esbbq_age -> age)
    df['category'] = df['task'].str.replace('esbbq_', '')
    
    # Pivot table: models as rows, categories as columns
    if 'accuracy' in df.columns:
        pivot_df = df.pivot_table(
            index='model',
            columns='category',
            values='accuracy',
            aggfunc='first'
        )
        
        print("\nAccuracy by Category:")
        display(pivot_df)
        
        # Save category breakdown
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        category_csv = f"{RESULTS_DIR}/mbbq_category_breakdown_{timestamp}.csv"
        pivot_df.to_csv(category_csv)
        print(f"\n💾 Category breakdown saved: {category_csv}")
    
    # Show bias score distribution if available
    bias_cols = [col for col in df.columns if 'bias' in col.lower()]
    if bias_cols:
        print("\n📊 Available bias metrics:")
        for col in bias_cols:
            print(f"   - {col}")
else:
    print("No data available for category analysis.")